# Визуализация алгоритма DES

In [1]:
import json
from functools import reduce


def get_bit(number, bit):
    return 1 & (number >> bit)

def rearrange_bits_by_matrix(number, matrix):
    return reduce(lambda a, i: (a << 1) | get_bit(number, i - 1), matrix, 0)

def split(number, mid_bit, parts):
    return reduce(lambda a, i: a + [(number >> (i * mid_bit)) & ((1 << mid_bit) - 1)], reversed(range(0, parts)), [])

def merge(numbers, mid_bit):
    return reduce(lambda a, i: (a << mid_bit) | (i & ((1 << mid_bit) - 1)), numbers, 0)

def cyclic_shift(number, bits, limit):
    return ((number << bits) | ((number << bits) >> limit)) & ((1 << limit) - 1)

In [2]:
with open("DES_tables.json") as DES_tables:
    tables = json.load(DES_tables)

In [3]:
def prepare_data_from_string(string, length):
    print(f"Preparing string: '{string}'")
    bts = bytearray(string, 'ascii')[:length // 8]
    print(" ".join(string), " ".join(str(i) for i in bts), " ".join(format(i, '08b') for i in bts), sep='\n')
    output = reduce(lambda a, i: (a << 8) + i, bts, 0)
    print(format(output, f'0{length}b'), output, "", sep='\n')
    return output

print("Input text: first and last names, 64bit")
plain_text = prepare_data_from_string("Name_Surname", 64)
print("Key: uni number and fathers name first letter, 56bit")
key = prepare_data_from_string("123456L", 56)

Input text: first and last names, 64bit
Preparing string: 'Name_Surname'
N a m e _ S u r n a m e
78 97 109 101 95 83 117 114
01001110 01100001 01101101 01100101 01011111 01010011 01110101 01110010
0100111001100001011011010110010101011111010100110111010101110010
5647915689857742194

Key: uni number and fathers name first letter, 56bit
Preparing string: '123456L'
1 2 3 4 5 6 L
49 50 51 52 53 54 76
00110001 00110010 00110011 00110100 00110101 00110110 01001100
00110001001100100011001100110100001101010011011001001100
13847469359445580



### Шаг 1: начальная перестановка данных

In [4]:
print("Input text initial permutation:\n", format(plain_text, '064b'), sep='\n')
prepared_text = rearrange_bits_by_matrix(plain_text, tables['ip'])
print("\nis changed to\n", format(prepared_text, '064b'), sep='\n')

Input text initial permutation:

0100111001100001011011010110010101011111010100110111010101110010

is changed to

1000110110101000011100110000000001111110101110100000111111111111


### Шаг 2: начальная перестановка ключа

In [5]:
print("Key permuted choice:\n", format(key, '056b'), sep='\n')
prepared_key = rearrange_bits_by_matrix(key, tables["pc1"])
print("\nis changed to\n", format(prepared_key, '056b'), sep='\n')

Key permuted choice:

00110001001100100011001100110100001101010011011001001100

is changed to

01010100001100100000111100010000000101111110011111100001


### Шаг 2: разделение и циклический сдвиг ключа

In [6]:
print("Key separation:\n", format(prepared_key, '056b'), sep='\n')
key_l, key_r = split(prepared_key, 28, 2)
print("\nis split to\n", f"{format(key_l, '028b')} {format(key_r, '028b')}", "\n", sep='\n')

shifted_keys = {}
print("Key shift (there are only two shift options, by 1 bit and by 2 bits):\n")
for i in set(tables["sls"]):
    print(f"{format(key_l, '028b')} {format(key_r, '028b')}", f"\nis shifted by {i} to\n", sep='\n')
    sh_key_l = cyclic_shift(key_l, i, 28)
    sh_key_r = cyclic_shift(key_r, i, 28)
    print(f"{format(sh_key_l, '028b')} {format(sh_key_r, '028b')}", "", sep='\n')
    shifted_keys[i] = (sh_key_l, sh_key_r)

Key separation:

01010100001100100000111100010000000101111110011111100001

is split to

0101010000110010000011110001 0000000101111110011111100001


Key shift (there are only two shift options, by 1 bit and by 2 bits):

0101010000110010000011110001 0000000101111110011111100001

is shifted by 1 to

1010100001100100000111100010 0000001011111100111111000010

0101010000110010000011110001 0000000101111110011111100001

is shifted by 2 to

0101000011001000001111000101 0000010111111001111110000100



### Шаг 3: генерация раундовых ключей

In [7]:
print("Key generation:\n")
keys = [None] * 16

for i in range(0, 16):
    print(f"Generating key for round {i}...")
    pre_key = merge([shifted_keys[tables["sls"][i]][0], shifted_keys[tables["sls"][i]][1]], 28)
    print("\nPermuted choice:\n", format(pre_key, '056b'), sep='\n')
    keys[i] = rearrange_bits_by_matrix(pre_key, tables["pc2"])
    print("\nis changed to\n", format(keys[i], '048b'), "\n", sep='\n')

Key generation:

Generating key for round 0...

Permuted choice:

10101000011001000001111000100000001011111100111111000010

is changed to

011000001001011001110101010110100000000110100100


Generating key for round 1...

Permuted choice:

10101000011001000001111000100000001011111100111111000010

is changed to

011000001001011001110101010110100000000110100100


Generating key for round 2...

Permuted choice:

01010000110010000011110001010000010111111001111110000100

is changed to

011000100011111001100110001111000001100001000110


Generating key for round 3...

Permuted choice:

01010000110010000011110001010000010111111001111110000100

is changed to

011000100011111001100110001111000001100001000110


Generating key for round 4...

Permuted choice:

01010000110010000011110001010000010111111001111110000100

is changed to

011000100011111001100110001111000001100001000110


Generating key for round 5...

Permuted choice:

01010000110010000011110001010000010111111001111110000100

is changed 

### Шаг 4: Функция f

In [8]:
def f(value, key, log):
    if log: print("Expanding and XORing with key R part:\n", format(value, '032b'), sep='\n')
    expanded_value = rearrange_bits_by_matrix(value, tables["e"])
    if log: print("\nexpanded to\n", format(expanded_value, '048b'), sep='\n')
    xor = expanded_value ^ key
    b_parts = split(xor, 6, 8)
    if log: print("\nXORed to\n", format(xor, '048b'), " ".join(format(i, '06b') for i in b_parts), "", sep='\n')
    nums = [None] * 8
    if log: print("Applying S boxes:\n", " ".join(format(i, '06b') for i in b_parts), sep='\n')
    for index, part in enumerate(b_parts):
        row = (get_bit(part, 5) << 1) | (get_bit(part, 0))
        col = reduce(lambda a, b: (a << 1) | b, [get_bit(part, i) for i in reversed(range(1, 5))], 0)
        nums[index] = tables["s"][index][row * 16 + col]
    if log: print("\nchanged to\n", " ".join(format(i, '04b') for i in nums), sep='\n')
    pre_result = merge(nums, 4)
    if log: print("\nconcatenated to\n", format(pre_result, '032b'), "", sep='\n')
    if log: print("R permutation:\n", format(pre_result, '032b'), sep='\n')
    result = rearrange_bits_by_matrix(pre_result, tables["p"])
    if log: print("\nchanged to\n", format(result, '032b'), sep='\n')
    return result

### Шаг 5: раунды шифрования

In [9]:
print("DES main body:\n")
L, R = split(prepared_text, 32, 2)

for i in range(0, 16):
    print(f"Round {i}...\n", f"L: {format(L, '032b')}, R: {format(R, '032b')}", "", sep='\n')
    old_R = R
    R = f(R, keys[i], True) ^ L
    L = old_R
    print("", f"L: {format(L, '032b')}, R: {format(R, '032b')}", "\n", sep='\n')

DES main body:

Round 0...

L: 10001101101010000111001100000000, R: 01111110101110100000111111111111

Expanding and XORing with key R part:

01111110101110100000111111111111

expanded to

011111111111111110100000001011111010101111111101

XORed to

000111110110100111010101011101011010101001011001
000111 110110 100111 010101 011101 011010 101001 011001

Applying S boxes:

000111 110110 100111 010101 011101 011010 101001 011001

changed to

0100 0110 0000 0010 1000 0111 0001 0000

concatenated to

01000110000000101000011100010000

R permutation:

01000110000000101000011100010000

changed to

10000000000111110000010100000100

L: 01111110101110100000111111111111, R: 00001101101101110111011000000100


Round 1...

L: 01111110101110100000111111111111, R: 00001101101101110111011000000100

Expanding and XORing with key R part:

00001101101101110111011000000100

expanded to

000100000000001101011101011101011011110110100000

XORed to

011100001001010100101000001011111011110000000100
011100 001001 

In [10]:
print("Final swap:\n")
L, R = R, L
print(f"L: {format(L, '032b')}, R: {format(R, '032b')}")
output_text = merge([L, R], 32)
print("", format(output_text, '064b'), sep='\n')

Final swap:

L: 01000100100000100001100010001100, R: 10100110100110000001011000010111

0100010010000010000110001000110010100110100110000001011000010111


### Шаг 6: финальная перестановка данных

In [11]:
print("Input text final permutation:\n", format(output_text, '064b'), sep='\n')
cipher_text = rearrange_bits_by_matrix(output_text, tables['ip-1'])
print("\nis changed to\n", format(cipher_text, '064b'), "", sep='\n')
print("Result:\n", format(cipher_text, '064b'), cipher_text, sep='\n')

Input text final permutation:

0100010010000010000110001000110010100110100110000001011000010111

is changed to

1000110100000010000000010111010010100100110100110101100101000000

Result:

1000110100000010000000010111010010100100110100110101100101000000
10160685309794408768


### Шаг 7: проверка

In [12]:
def compare(check, real):
    return "correct" if check == real else "wrong"

print("DES result:", format(cipher_text, '064b'))
ch_output_text = rearrange_bits_by_matrix(cipher_text, tables['ip'])
print("Output text:", format(ch_output_text, '064b'), compare(ch_output_text, output_text), end='\n\n')

ch_L, ch_R = split(ch_output_text, 32, 2)
print(f"L: {format(ch_L, '032b')}, R: {format(ch_R, '032b')}", end='\n\n')
ch_L, ch_R = ch_R, ch_L

for i in range(0, 16):
    old_L = ch_L
    ch_L = f(ch_L, keys[15 - i], False) ^ ch_R
    ch_R = old_L
    print(f"Round {i}: L = {format(ch_L, '032b')}, R = {format(ch_R, '032b')}")

ch_prepared_text = merge([ch_L, ch_R], 32)
print(f"\nPrepared text: {format(ch_prepared_text, '064b')}", compare(ch_prepared_text, prepared_text), end='\n\n')

ch_plain_text = rearrange_bits_by_matrix(ch_prepared_text, tables['ip-1'])
print("Plain text:", format(ch_plain_text, '064b'), compare(ch_plain_text, plain_text), end='\n\n')

ch_letters = bytearray(split(ch_plain_text, 8, 8))
print(f"Decoded string value: '{ch_letters.decode('ascii')}'")

DES result: 1000110100000010000000010111010010100100110100110101100101000000
Output text: 0100010010000010000110001000110010100110100110000001011000010111 correct

L: 01000100100000100001100010001100, R: 10100110100110000001011000010111

Round 0: L = 11000000011110100010011111011010, R = 10100110100110000001011000010111
Round 1: L = 11001011010010000001110010100101, R = 11000000011110100010011111011010
Round 2: L = 01000101001010111000011000101111, R = 11001011010010000001110010100101
Round 3: L = 11000011111110011011111010101111, R = 01000101001010111000011000101111
Round 4: L = 01110111100111101000101010110010, R = 11000011111110011011111010101111
Round 5: L = 10100011101111011001011000011111, R = 01110111100111101000101010110010
Round 6: L = 01110010010111100001100100001110, R = 10100011101111011001011000011111
Round 7: L = 10010000001010100100110101000101, R = 01110010010111100001100100001110
Round 8: L = 01000101011001101000101010100111, R = 10010000001010100100110101000101
Round 